# Region Search Overview and Examples
This notebook shows how to query MAST by region and shows how that could be used in an RRN workflow.
## Imports

In [ ]:
from astropy import units as u
from astroquery.mast import Observations
from pyvo.dal import TAPService

from mast_aladin import AppSidecar, MastAladin

## MAST Region Search Overview
Region search in MAST is supported via the [MAST CAOM TAP service](https://mast.stsci.edu/vo-tap/api/v0.1/caom/).  We can use the `pyvo` package to make the TAP queries with the query constraints specified in SQL-like [Astronomical Data Query Language (ADQL)](https://www.ivoa.net/documents/ADQL/20231215/index.html).

### Basic MAST TAP Query Using pyvo
Given the ADQL for a TAP query, that TAP query can be performed on the MAST TAP service with the following code:
```python
from pyvo.dal import TAPService

# Create your ADQL.
adql = 'SELECT * FROM...whatever your query is'

# Create the TAPService object by supplying the endpoint of the MAST CAOM TAP service.
mast_caom_tap_service = TAPService('https://mast.stsci.edu/vo-tap/api/v0.1/caom')

# Perform the query.  "run_sync" allows for a longer-running query by managing job creation and polling 
# behind the scenes.  To the user here this is a synchronous call.
results = mast_caom_tap_service.run_async(adql)

# Convert the results from a TAPResults object to an astropy table.
table_results = results.to_table()
```

### ADQL Structure for a Region Query
To search MAST for the first `limit` observations whose footprints overlap a given region, the basic ADQL looks like:
```sql
SELECT TOP {limit} *
FROM dbo.obspointing o
WHERE
CONTAINS(POINT('ICRS',s_ra,s_dec), 
   {adql_region}
)=1
```
where `adql_region` is the ADQL syntax for specifying a polygon or circle.  Those specification require a coordinate frame (which we always specify as `'ICRS'`), and a list of numbers (decimal degrees).  For polygons, the list of numbers is RA1, Dec1, RA2, Dec2, etc.  For circles the numbers are RA, Dec and radius.

#### ADQL Example: Query top 10 observations in a 1.0 x 0.2 degree rectangle around 270, 60
```sql
SELECT TOP 10 *
FROM dbo.obspointing o
WHERE
CONTAINS(POINT('ICRS',s_ra,s_dec), 
   POLYGON('ICRS', 270.5, 60.1, 269.5, 60.1, 269.5, 59.9, 270.5, 59.9)
)=1
```

#### ADQL Example: Query top 50 observations in a 0.2 degree radius circle around 345, -50
```sql
SELECT TOP 50 *
FROM dbo.obspointing o
WHERE
CONTAINS(POINT('ICRS',s_ra,s_dec), 
   CIRCLE('ICRS', 345, -50, 0.2)
)=1
```



### Helper Functions
The ADQL for a region query is a bit involved, so we'll provide some helper functions to make the region part of the query easier to construct.  They'll start out in this notebook, but belong somewhere else like `MastAladin` or `astroquery`.

The user would typically call one of:

- `mast_reqion_query(region, other_clauses='', limit=50000)` 
- `mast_region_query_aladin_fov(aladin, other_clauses='', limit=50000, show_query=True)`

which would use the other two helper functions:
- `create_mast_region_adql(region, other_clauses='', limit=50000)`
- `create_adql_region(region)`

So far for demo purposes, `region` can be:
- an `s_region`
- list of points (RA/Dec tuples)
But it should be extended to support:
- astropy regions
- AIDA fovs
- Other?

In [ ]:
def create_adql_region(region):
    """
    Returns the ADQL description of the given polygon or cirle region
    
    region - Iterable of RA/Dec pairs as lists/sequences OR
             stc-s POLYGON or CIRCLE string OR
             astropy region (TBD) OR
             other (TBD)
    """
    adql_region = None
    if isinstance(region, str):
        region = region.lower()
        parts = region.split()
        if parts[0] == 'polygon':
            # Assume we've got an STC-s polygon
            try:
                float(parts[1])
                point_parts = parts[1:] # There's no coord frame present if we got this far.
            except ValueError:
                point_parts = parts[2:]
            point_string = ','.join(point_parts)
            adql_region = f"POLYGON('ICRS',{point_string})"
        elif parts[0] == 'circle':
            # Assume we've got an STC-s circle
            try:
                float(parts[1])
                radius = parts[1]
                point_parts = parts[2:]
            except ValueError:
                radius = parts[2]
                point_parts = parts[3:]
            point_string = ','.join(point_parts)
            adql_region = f"CIRCLE('ICRS',{point_string},{radius})"
    else:
        # Assume we've got a list/sequence of points (lists/sequences)
        point_string = ','.join([str(num) for point in region for num in point])
        adql_region = f"POLYGON('ICRS',{point_string})"

    return adql_region

In [ ]:
def create_mast_region_adql(region, other_clauses='', limit=50000):
    """
    Creates the ADQL to query the MAST CAOM TAP service for observations over the
    specified region.
    
    region - Iterable of RA/Dec pairs as lists/sequences OR
             stc-s POLYGON or CIRCLE string OR
             astropy region (TBD) OR
             other (TBD)
    other_clauses - ADQL to tack on to the end of the CAOM TAP obspointing query
    limit - max number of rows to return (default 50000)

    Returns a string containing the necessary ADQL.
    """
    adql = None
    adql_region = create_adql_region(region)
    if adql_region:
        adql = f"""
SELECT TOP {limit} *
FROM dbo.obspointing o
WHERE
CONTAINS(POINT('ICRS',s_ra,s_dec), 
   {adql_region}
)=1
"""
        adql += other_clauses
        return adql

In [ ]:
def mast_reqion_query(region, other_clauses='', limit=50000, show_query=True):
    """
    Query MAST observations over the specified region using the MAST CAOM TAP service.
    
    region - Iterable of RA/Dec pairs as lists/sequences OR
             stc-s POLYGON or CIRCLE string OR
             astropy region (TBD) OR
             other (TBD)
    other_clauses - ADQL to tack on to the end of the CAOM TAP obspointing query
    limit - max number of rows to return (default 50000)

    Returns an astropy table of MAST observations much like astroquery.mast Observations queries.
    """
    adql = create_mast_region_adql(region, other_clauses, limit)
    if not adql:
        raise ValueError('region must be iterable of RA/Dec pairs as lists/sequences '
                         'OR valid stc-s POLYGON or CIRCLE string')

    mast_caom_tap_service = TAPService("https://mast.stsci.edu/vo-tap/api/v0.1/caom")
    
    if show_query:
        print(f'Querying MAST CAOM TAP with:\n{adql}')
    
    results = mast_caom_tap_service.run_async(adql)
    table_results = results.to_table()
    if show_query:
        print(f'Number of results: {len(table_results)}')
    
    return table_results

In [ ]:
def mast_region_query_aladin_fov(aladin, other_clauses='', limit=50000, show_query=True):
    """
    Query MAST observations over the FoV of the specified Aladin instance.  We won't do the
    search if Aladin's FoV is greater than 1 degree or its WCS doesn't exist for some reason.
    
    aladin - A MastAladin instance.
    other_clauses - ADQL to tack on to the end of the CAOM TAP obspointing query
    limit - max number of rows to return (default 50000)

    Returns an astropy table of MAST observations much like astroquery.mast Observations queries.
    """
    results = None
    
    # Don't do it if FoV is bigger than 1 degree.
    if aladin.fov > 1 * u.degree:
        print(f'FoV of {aladin.fov} is greater than the 1 degree limit.')
    elif aladin.wcs:
        footprint = aladin.wcs.calc_footprint()
        results = mast_reqion_query(footprint, other_clauses=other_clauses, limit=limit, show_query=show_query)
    else:
        print('Aladin WCS information not found')

    return results

# Example: Search on Existing Footprint
Given the footprint of a JWST observation, search MAST for HST observations whose footprints overlap the JWST observation.
## Set Up Aladin
Create Aladin in a sidecar centered on M101.

In [ ]:
aladin = MastAladin(
    target="M101",
    fov=0.5
)
apps_initialized = AppSidecar.open(aladin, anchor='split-right')
AppSidecar.resize_all(800)

## Find and Display Desired JWST Observation Footprint
This example uses `astroquery.mast` observation search to find the footprints, but it could have come from JdaViz or many other places.

After finding the observation via `astroquery.mast` we add the result table as an overlay in Aladin so that we can see the footprint (in green).

In [ ]:
jwst_obs = Observations.query_criteria(obs_id='jw03429-o013_t005_nircam_clear-f444w')
jwst_obs_layer = aladin.add_table(jwst_obs, name='JWST Observation', color='green')

## Search on the Existing Footprint
Use the existing footprint as the `region` input for a MAST region TAP query.

This example adds the `other_clauses` argument to limit the results to HST.  It does not bother to specify a `limit` since there aren't likely to be that many results.

Add the results as an Aladin overlay (in Portal footprint orange).

In [ ]:
# Search MAST for HST observations that overlap with the JWST observation.
footprint = jwst_obs[0]['s_region']
other_clauses = """
AND obs_collection = 'HST'
AND project = 'HST'
"""

hst_obs = mast_reqion_query(footprint, other_clauses=other_clauses)
hst_obs_layer = aladin.add_table(hst_obs, name='HST Observations', color='#FF6600')

# Example: Search on Current Aladin FoV
Search on the polygon defined by Aladin's current viewport.  The polygon corners are computed using Aladin's notion of the WCS for the viewport.  We don't allow searches if the FoV is greater than 1 degree.

We'll use the existing Aladin instance.

## Search for HST on Current FoV.

In [ ]:
# Search MAST for HST observations that contain the current Aladin FoV.
other_clauses = """
AND obs_collection = 'HST'
AND project = 'HST'
"""
hst_fov_obs = mast_region_query_aladin_fov(aladin, other_clauses=other_clauses)

# Add the results in yellow.
hst_fov_obs_layer = aladin.add_table(hst_fov_obs, name='HST FoV Observations', color='yellow')

## Search for TESS on Current FoV.

In [ ]:
# Search MAST for TESS observations that contain the current Aladin FoV.
other_clauses = """
AND obs_collection = 'TESS'
AND project = 'TESS'
"""
tess_fov_obs = mast_region_query_aladin_fov(aladin, other_clauses=other_clauses)

# Add the results in blue.
tess_fov_obs_layer = aladin.add_table(tess_fov_obs, name='TESS FoV Observations', color='blue')